In [1]:
# ENV["PYTHON"]= "/kuacc/users/oulas15/.conda/envs/oulas15/bin/python"
# using Pkg
# Pkg.build("PyCall")

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/299304989a5e6473d985212c28928899c74e9421/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/169bb8ea6b1b143c5cf57df6d34d022a7b60c6db/build.log`
Precompiling project...
  ✓ PyCall
  ✓ MLDatasets
  2 dependencies successfully precompiled in 51 seconds (225 already precompiled)


In [1]:
using Knet, CUDA, MLDatasets, ArgParse, Images, Random, Statistics,Base, Plots,NPZ
#include(Knet.dir("data","imagenet.jl"))
const F = Float32
Atype = CUDA.functional() ? KnetArray{F} : Array{F}

KnetArray{Float32, N} where N

In [2]:
train_data = npzread("../Mouse_n20/train/train_data.npz")

Dict{String, Array{T, 3} where T} with 4 entries:
  "X_val"   => [15.1078 0.766439 … 20.7861 -12.73; -10.8562 -2.43127 … 28.4208 …
  "Y_val"   => UInt16[0x0000 0x0000 … 0x0000 0x0000; 0x0000 0x0000 … 0x0000 0x0…
  "X_train" => [-8.65205 -38.3173 … -9.87815 3.37247; -35.7896 -8.26069 … -16.3…
  "Y_train" => UInt16[0x0000 0x0000 … 0x0000 0x0000; 0x0000 0x0000 … 0x0000 0x0…

In [3]:
test_data = npzread("../Mouse_n20/test/test_data.npz")

Dict{String, Array{T, 3} where T} with 2 entries:
  "Y_test" => UInt16[0x0000 0x0000 … 0x0000 0x0000; 0x0000 0x0000 … 0x0000 0x00…
  "X_test" => [-17.5679 -6.04057 … -53.2153 -15.2519; -6.9902 -4.6863 … 6.61619…

In [4]:
xtrn=train_data["X_train"];
xval=train_data["X_val"];
yval=train_data["Y_val"]
yval=Array{F}(yval)
xtrn=permutedims(xtrn, [2, 3, 1]);
xval=permutedims(xval, [2, 3, 1]);
yval=permutedims(yval, [2, 3, 1]);
println(size(xtrn))
println(size(xval))
println(size(yval))

(128, 128, 908)
(128, 128, 160)
(128, 128, 160)


In [5]:
#plot(Matrix{Gray{Float32}}(xval[:,:,1]./255.0))

In [6]:
data = cat(xtrn,xval,dims=3);

In [7]:
data_mean = mean(data);
data_std = stdm(data,data_mean);

In [8]:
batch_size = 32
dtrn = minibatch(xtrn, batch_size; xsize = (128, 128, 1,:), xtype = Atype)
dval = minibatch(xval,yval, 32; xsize = (128, 128, 1,:), xtype = Atype);

In [22]:
function encode(ϕ, x)
    #x = reshape(x, (128,128,1,:))
    x = conv4(ϕ[1], x, padding=1)
    x = relu.(x .+ ϕ[2])
    x = conv4(ϕ[3], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[4])
    x = pool(x, stride = 2, mode=0)
    #println(size(x))
    x = conv4(ϕ[5], x, padding=1)
    x = relu.(x .+ ϕ[6])
    x = conv4(ϕ[7], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[8])
    x = pool(x, stride = 2, mode=0)
    #println(size(x))
    x = conv4(ϕ[9], x, padding=1)
    x = relu.(x .+ ϕ[10])
    x = conv4(ϕ[11], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[12])
    x = pool(x, stride = 2, mode=0)
    #println(size(x))
    
    mu = conv4(ϕ[13], x, padding=1, stride=1)
    mu = mu .+ ϕ[14]
    
    logvar = conv4(ϕ[15], x, padding=1, stride=1)
    logvar = logvar .+ ϕ[16]
    
    return mu, logvar
end

function decode(θ, z)
    z = conv4(θ[1], z, padding=1)
    z = relu.(z .+ θ[2])
    z = conv4(θ[3], z, padding=1, stride=1)
    z = relu.(z .+ θ[4])
    z = deconv4(θ[5], z, padding=0, stride=2)
    z = relu.(z .+ θ[6])
    #println(size(z))

    z = conv4(θ[7], z, padding=1)
    z = relu.(z .+ θ[8])
    z = conv4(θ[9], z, padding=1, stride=1)
    z = relu.(z .+ θ[10]) 
    z = deconv4(θ[11], z, padding=0, stride=2)
    z = relu.(z .+ θ[12])
    
    #println(size(z))
    z = conv4(θ[13], z, padding=1)
    z = relu.(z .+ θ[14])
    z = conv4(θ[15], z, padding=1, stride=1)
    z = relu.(z .+ θ[16]) 
    z = deconv4(θ[17], z, padding=0, stride=2)
    z = relu.(z .+ θ[18])
    #println(size(z))
    z = conv4(θ[19], z, padding=1)
    z = z .+ θ[20]
    return z
end



function weights(channel)
    
    θ = [] # z->x

    push!(θ, xavier(3, 3, channel*2, channel*4))
    push!(θ, zeros(1, 1, channel*4, 1))

    push!(θ, xavier(3,3,channel*4,channel*4))
    push!(θ, zeros(1, 1, channel*4, 1))
    
    push!(θ, xavier(2, 2, channel*4, channel*4))
    push!(θ, zeros(1,1,channel*4,1))
    
    
    push!(θ, xavier(3, 3, channel*4, channel*2))
    push!(θ, zeros(1,1, channel*2,1))
    
    push!(θ, xavier(3,3,channel*2,channel*2))
    push!(θ, zeros(1, 1, channel*2, 1))
    
    push!(θ, xavier(2,2,channel*2,channel*2))
    push!(θ, zeros(1, 1, channel*2, 1))
    
    
    push!(θ, xavier(3, 3, channel*2, channel))
    push!(θ, zeros(1,1, channel,1))
    
    push!(θ, xavier(3,3,channel,channel))
    push!(θ, zeros(1, 1, channel, 1))
    
    push!(θ, xavier(2,2,channel,channel))
    push!(θ, zeros(1, 1, channel, 1))
    

    push!(θ, xavier(3,3,channel,1))
    push!(θ, zeros(1, 1, 1, 1))
    
    
    θ = map(a->convert(Atype,a), θ)

    ϕ = [] # x->z

    push!(ϕ, xavier(3, 3, 1, channel))
    push!(ϕ, zeros(1, 1, channel, 1))

    push!(ϕ, xavier(3, 3, channel, channel))
    push!(ϕ, zeros(1, 1, channel, 1))
    
    
    push!(ϕ, xavier(3, 3, channel, channel*2))
    push!(ϕ, zeros(1,1, channel*2, 1))

    push!(ϕ, xavier(3, 3, channel*2,channel*2))
    push!(ϕ, zeros(1, 1, channel*2, 1))
    
    
    push!(ϕ, xavier(3, 3, channel*2, channel*4))
    push!(ϕ, zeros(1,1, channel*4, 1))

    push!(ϕ, xavier(3, 3, channel*4, channel*4))
    push!(ϕ, zeros(1, 1, channel*4, 1))
    
    
    push!(ϕ, xavier(3, 3, channel*4, channel*2))
    push!(ϕ, zeros(1, 1, channel*2, 1))
    
    push!(ϕ, xavier(3, 3, channel*4, channel*2))
    push!(ϕ, zeros(1, 1, channel*2, 1))

    ϕ = map(a->convert(Atype,a), ϕ)
    return θ, ϕ
end

weights (generic function with 1 method)

In [23]:
function KLD(mu, logvar)
    kl_error = -0.5 * sum(1 .+ (logvar - mu.*mu - exp.(logvar)))
    return kl_error
end

KLD (generic function with 1 method)

In [24]:
function MSE(x, y,gaus_std,data_std)
    return mean((x - y).^2) / (2.0*(gaus_std/data_std)^2)
end

MSE (generic function with 1 method)

In [25]:
function reparameterize(mu,logvar)
    std = exp.(logvar.*0.5)
    epsilon = convert(Atype, randn(F, size(mu)))
    z = mu .+ (epsilon.*std)
    return z 
end

reparameterize (generic function with 1 method)

In [26]:
wdec, wenc = weights(32);
w = Dict(
        :encoder => wenc,
        :decoder => wdec)
opt = Dict(
        :encoder => map(wi->Knet.Adam(lr = 1e-3), w[:encoder]),
        :decoder => map(wi->Knet.Adam(lr = 1e-3), w[:decoder]),
    )

Dict{Symbol, Vector{Adam}} with 2 entries:
  :decoder => [Adam(0.001, 0.9, 0.999, 1.0e-8, 0, 0.0, nothing, nothing), Adam(…
  :encoder => [Adam(0.001, 0.9, 0.999, 1.0e-8, 0, 0.0, nothing, nothing), Adam(…

In [35]:
# x = first(dtrn)
# x_normalized = (x .- data_mean)./ data_std 
# mu, logvar = encode(wenc, x_normalized)

(64, 64, 32, 8)
(32, 32, 64, 8)
(16, 16, 128, 8)


(K32(16,16,64,8)[-0.016579967⋯], K32(16,16,64,8)[-0.007940645⋯])

In [36]:
# z = reparameterize(mu,logvar)
# y = decode(wdec,z);

(32, 32, 128, 8)
(64, 64, 64, 8)
(128, 128, 32, 8)


In [27]:
function loss(w, x,gaus_std,data_mean,data_std,epoch)
    wdec, wenc = w[:decoder], w[:encoder]
    x_normalized = (x .- data_mean)./ data_std
    mu, logvar = encode(wenc, x_normalized)
    z = reparameterize(mu,logvar)
    y = decode(wdec,z)
    #y = (y .* data_std) .+ data_mean  
    #println(KLD(mu,logvar))
    #if epoch<20
    #    return MSE(x_normalized,y,gaus_std,data_std) #+ (KLD(mu,logvar)/(128*128*8))
    #else
    return MSE(x_normalized,y,gaus_std,data_std) + (KLD(mu,logvar)/(128*128*32))
    #end
end
lossgradient = grad(loss);

In [28]:
function val_mse(w, x,gaus_std,data_mean,data_std)
    wdec, wenc = w[:decoder], w[:encoder]
    x_normalized = (x .- data_mean)./ data_std
    mu, logvar = encode(wenc, x_normalized)
    z = reparameterize(mu,logvar)
    y = decode(wdec,z)
    #y = (y .* data_std) .+ data_mean  
    return MSE(x_normalized,y,gaus_std,data_std), KLD(mu,logvar)/(128*128*32)
end

val_mse (generic function with 1 method)

In [29]:
g_std = 20

20

In [30]:
x = first(dtrn);

In [31]:
val_mse(w, x,g_std,data_mean,data_std)

(2.519552124748743, 0.0012312226463109255)

In [43]:
dw = lossgradient(w, x,g_std,data_mean,data_std)

(64, 64, 32, 8)
(32, 32, 64, 8)
(16, 16, 128, 8)
(32, 32, 128, 8)
(64, 64, 64, 8)
(128, 128, 32, 8)


Dict{Symbol, Vector{KnetArray{Float32, 4}}} with 2 entries:
  :decoder => [K32(3,3,64,128)[7.829876e-6⋯], K32(1,1,128,1)[0.00032571968⋯], K…
  :encoder => [K32(3,3,1,32)[-0.00034068868⋯], K32(1,1,32,1)[0.0007768106⋯], K3…

In [20]:
tp = first(dval)
x_,y_ = tp;

In [32]:
number_of_epochs = 200
for epoch in 1:number_of_epochs
   println("Epoch",epoch)
    for (i,x)  in enumerate(dtrn)
        dw = lossgradient(w, x,g_std,data_mean,data_std,epoch)
        update!(w, dw, opt)
    end
    ms,kld  = val_mse(w,x_,g_std,data_mean,data_std) 
    println(ms)
    println(kld)
end

Epoch1
2.3187509060740727
1.8124051166523714e-6
Epoch2
2.316062555725244
4.851531411986798e-5
Epoch3
1.4697366810440764
0.038274772465229034
Epoch4
0.9981796786205379
0.051245033740997314
Epoch5
0.8493745332461279
0.06040411815047264
Epoch6
0.8133631097466701
0.060173433274030685
Epoch7
0.7677236439338035
0.06915175914764404
Epoch8
0.7400181705470934
0.06891046464443207
Epoch9
0.7239625061984765
0.07285068929195404
Epoch10
0.7090624056071109
0.07093406468629837
Epoch11
0.7108011209630495
0.07088851928710938
Epoch12
0.6943939413149016
0.07296082377433777
Epoch13
0.6740898215750659
0.07264506816864014
Epoch14
0.6637041117609687
0.07422909140586853
Epoch15
0.6546597580182534
0.07201103121042252
Epoch16
0.6576374020353992
0.0687367245554924
Epoch17
0.6698055904239368
0.06766985356807709
Epoch18
0.6330734833707147
0.06651546061038971
Epoch19
0.6233244209914182
0.05868515372276306
Epoch20
0.6158974940527505
0.05728546530008316
Epoch21
0.6078707022617039
0.05581868439912796
Epoch22
0.60649323

LoadError: DimensionMismatch("Error converting batch from 16384×908 Matrix{Float64} to KnetArray{Float32, N} where N: InterruptException()")

In [35]:
dw_

Dict{Symbol, Vector{KnetArray{Float32, 4}}} with 2 entries:
  :decoder => [K32(3,3,64,64)[-0.12707093⋯], K32(1,1,64,1)[-0.031114511⋯], K32(…
  :encoder => [K32(3,3,1,64)[-0.1059588⋯], K32(1,1,64,1)[-0.25156972⋯], K32(3,3…